## Import Model

In [7]:
# !gdown 1jwJqh3CA6Grcykoh-AA8LlUfyamdFuty

In [1]:
# !pip install sklearn_crfsuite
import joblib
import pandas as pd
model = joblib.load("model.joblib")

In [9]:
stopwords = ["ผู้", "ที่", "ซึ่ง", "อัน"]

def tokens_to_features(tokens, i):
  word = tokens[i]

  features = {
    "bias": 1.0,
    "word.word": word,
    "word[:3]": word[:3],
    "word.isspace()": word.isspace(),
    "word.is_stopword()": word in stopwords,
    "word.isdigit()": word.isdigit(),
    "word.islen5": word.isdigit() and len(word) == 5
  }

  if i > 0:
    prevword = tokens[i - 1]
    features.update({
      "-1.word.prevword": prevword,
      "-1.word.isspace()": prevword.isspace(),
      "-1.word.is_stopword()": prevword in stopwords,
      "-1.word.isdigit()": prevword.isdigit(),
    })
  else:
    features["BOS"] = True

  if i < len(tokens) - 1:
    nextword = tokens[i + 1]
    features.update({
      "+1.word.nextword": nextword,
      "+1.word.isspace()": nextword.isspace(),
      "+1.word.is_stopword()": nextword in stopwords,
      "+1.word.isdigit()": nextword.isdigit(),
    })
  else:
    features["EOS"] = True

  return features

def parse(text):
  tokens = text.split()
  features = [tokens_to_features(tokens, i) for i in range(len(tokens))]
  return model.predict([features])[0]

In [10]:
parse("นายสมชาย เข็มกลัด 254 ถนนพญาไท วังใหม่ ปทุมวัน กรุงเทพมหานคร 10330")

array(['O', 'O', 'ADDR', 'ADDR', 'ADDR', 'LOC', 'LOC', 'POST'],
      dtype=object)

In [11]:
parse("นายมงคล 123/4 ตำบล บ้านไกล อำเภอ เมือง จังหวัด ลพบุรี 15000")

array(['ADDR', 'ADDR', 'ADDR', 'ADDR', 'ADDR', 'ADDR', 'ADDR', 'ADDR',
       'POST'], dtype=object)

## Using Random from ChatGPT

In [12]:
import random
import pandas as pd

# Sample Thai names and surnames
first_names = ["สมชาย", "วิชัย", "สมศักดิ์", "กิตติ", "อัศวิน", "ประสิทธิ์", "สุริยะ", "ชัยวัฒน์", "วัฒนา", "เอกชัย", "พัฒน์พงศ์", "สุพจน์", "วิเชียร", "อรุณ", "กำธร"]
last_names = ["มีสุข", "สวัสดี", "สุขใจ", "ใจดี", "ใจบุญ", "กิตติกูล", "ชนะพงศ์", "สุวรรณ", "คงเจริญ", "เพิ่มพูน", "เจริญสุข", "ชัยรัตน์", "ทรงชัย", "สุทธิชัย", "รุ่งเรือง"]

# Sample Thai locations
districts = ["สามย่าน", "ลาดพร้าว", "บางนา", "บางเขน", "ห้วยขวาง", "บางกะปิ", "ดอนเมือง", "บางบัวทอง", "บางพลี", "พระโขนง", "พญาไท", "บางกอกน้อย", "บางกอกใหญ่", "ปทุมวัน", "สาทร"]
subdistricts = ["ทุ่งมหาเมฆ", "สวนหลวง", "ลาดยาว", "สีกัน", "บางรัก", "ปากเกร็ด", "บางมด", "ลาดพร้าว", "ศาลายา", "บางกะปิ", "คลองตัน", "พระโขนง", "ลาดพร้าว", "บางนา", "บางซื่อ"]
provinces = ["กรุงเทพฯ", "กทม", "กรุงเทพมหานคร", "นนทบุรี", "ปทุมธานี", "สมุทรปราการ", "นครปฐม", "ชลบุรี", "อยุธยา", "สระบุรี", "ราชบุรี", "อ่างทอง"]
postal_codes = ["10100", "10240", "10120", "10230", "10310", "10150", "10210", "11120", "10270", "10540"]

# Function to generate random addresses with shuffled order (except name-surname order)
def generate_address(num_addresses):
    addresses = []
    tags = []
    
    for _ in range(num_addresses):
        # Generate name and surname, which must appear together
        first_name = random.choice(first_names)
        last_name = random.choice(last_names)
        name = f"นาย{first_name} {last_name}"
        name_tag = ["O", "O"]  # Tag '0' for first name and 'O' for last name
        
        # Other components for the address and their tags
        other_components = [
            (f"{random.randint(1, 999)}/{random.randint(1, 99)}", "ADDR"),
            (random.choice(districts), "LOC"),
            (random.choice(subdistricts), "LOC"),
            (random.choice(provinces), "LOC"),
            (random.choice(postal_codes), "POST")
        ]
        
        # Shuffle the other components
        random.shuffle(other_components)
        
        # Separate components and their tags after shuffling
        shuffled_components, shuffled_tags = zip(*other_components)
        
        # Combine name (first + last) with other shuffled components
        address = f"{name} " + " ".join(shuffled_components)
        address_tags = name_tag + list(shuffled_tags)
        
        # Append results
        addresses.append(address)
        tags.append(address_tags)
    
    return addresses, tags

# Generate 100 random addresses and their tags
random_addresses, random_tags = generate_address(100)

# Convert the generated addresses and tags to a DataFrame for CSV export
df_addresses = pd.DataFrame({
    "Address": random_addresses,
    "Tags": random_tags
})

# Display the DataFrame
df_addresses

,Address,Tags
0,นายอรุณ สุขใจ 10540 ปากเกร็ด 728/85 บางพลี อ่า...,"[O, O, POST, LOC, ADDR, LOC, LOC]"
1,นายอัศวิน ชนะพงศ์ บางบัวทอง นนทบุรี 10210 ปากเ...,"[O, O, LOC, LOC, POST, LOC, ADDR]"
2,นายวิเชียร ชนะพงศ์ 978/77 10540 ปทุมธานี สาทร ...,"[O, O, ADDR, POST, LOC, LOC, LOC]"
3,นายกิตติ เจริญสุข สวนหลวง บางเขน กรุงเทพฯ 300/...,"[O, O, LOC, LOC, LOC, ADDR, POST]"
4,นายสุริยะ เจริญสุข กรุงเทพฯ บางกะปิ 10230 บางน...,"[O, O, LOC, LOC, POST, LOC, ADDR]"
...,...,...
95,นายประสิทธิ์ มีสุข 10270 กรุงเทพมหานคร บางกอกน...,"[O, O, POST, LOC, LOC, LOC, ADDR]"
96,นายเอกชัย มีสุข 690/31 ลาดพร้าว 10120 ชลบุรี พ...,"[O, O, ADDR, LOC, POST, LOC, LOC]"
97,นายอรุณ ทรงชัย บางซื่อ บางกะปิ กรุงเทพฯ 352/77...,"[O, O, LOC, LOC, LOC, ADDR, POST]"
98,นายสมศักดิ์ ใจบุญ 290/39 บางกอกน้อย 10230 พระโ...,"[O, O, ADDR, LOC, POST, LOC, LOC]"


In [13]:
rows = df_addresses.shape[0]

for index, row in df_addresses.iterrows():
    content = df_addresses.iloc[index,0]
    df_addresses['Predict'] = df_addresses['Address'].apply(parse)

df_addresses.head()

,Address,Tags,Predict
0,นายอรุณ สุขใจ 10540 ปากเกร็ด 728/85 บางพลี อ่า...,"[O, O, POST, LOC, ADDR, LOC, LOC]","[O, O, POST, O, O, O, O]"
1,นายอัศวิน ชนะพงศ์ บางบัวทอง นนทบุรี 10210 ปากเ...,"[O, O, LOC, LOC, POST, LOC, ADDR]","[O, O, O, O, POST, O, O]"
2,นายวิเชียร ชนะพงศ์ 978/77 10540 ปทุมธานี สาทร ...,"[O, O, ADDR, POST, LOC, LOC, LOC]","[O, LOC, LOC, POST, LOC, LOC, O]"
3,นายกิตติ เจริญสุข สวนหลวง บางเขน กรุงเทพฯ 300/...,"[O, O, LOC, LOC, LOC, ADDR, POST]","[O, LOC, LOC, LOC, LOC, LOC, POST]"
4,นายสุริยะ เจริญสุข กรุงเทพฯ บางกะปิ 10230 บางน...,"[O, O, LOC, LOC, POST, LOC, ADDR]","[O, O, O, O, POST, O, O]"


In [14]:
from sklearn.metrics import confusion_matrix
import numpy as np

# Flatten the 'Tags' and 'Predict' columns to compare corresponding elements
true_tags = [tag for tags in df_addresses["Tags"] for tag in tags]
predicted_tags = [tag for tags in df_addresses["Predict"] for tag in tags]

# Create the confusion matrix
cm = confusion_matrix(true_tags, predicted_tags)

# Get the unique labels (tags) to display the confusion matrix with proper labels
labels = np.unique(true_tags)

# Convert the confusion matrix into a DataFrame for better visualization
cm_df = pd.DataFrame(cm, index=labels, columns=labels)

In [15]:
display(cm_df)

,ADDR,LOC,O,POST
ADDR,4,29,67,0
LOC,10,130,160,0
O,2,31,167,0
POST,1,0,0,99


In [58]:
import random

# Generate a fixed set of random numbers with a given seed
def generate_fixed_number_set(seed=42):
    random.seed(seed)  # Set a fixed seed for reproducibility of the number set
    number_set = random.sample(range(1, 101), 100)  # Generate a set of 10 unique random numbers
    return number_set

# Store the generated fixed set
fixed_number_set = generate_fixed_number_set()

def get_random_number():
    # Select a random number from the fixed set without setting a seed for the choice
    return random.choice(fixed_number_set)

# Example usage: calling multiple times
print(get_random_number())
print(get_random_number())
print(get_random_number())

14
15
87


In [132]:
# Updated function with 'rd' parameter to control shuffling
def generate_address(num_addresses, seed=42, rd=True):
    random.seed(seed)
    addresses = []
    tags = []
    
    for _ in range(num_addresses):
        # Generate name and surname, which must appear together
        first_name = random.choice(first_names)
        last_name = random.choice(last_names)
        name = f"นาย{first_name} {last_name}"
        name_tag = ["O", "O"]  # Tag 'O' for first name and last name
        
        # Other components for the address and their tags
        other_components = [
            (f"{random.randint(1, 999)}/{random.randint(1, 99)}", "ADDR"),
            (random.choice(districts), "LOC"),
            (random.choice(subdistricts), "LOC"),
            (random.choice(provinces), "LOC"),
            (random.choice(postal_codes), "POST")
        ]
        
        # Conditionally shuffle the other components based on the rd parameter
        if rd:
            random.shuffle(other_components)
        
        # Separate components and their tags (in either shuffled or fixed order)
        components, component_tags = zip(*other_components)
        
        # Combine name (first + last) with other components
        address = f"{name} " + " ".join(components)
        address_tags = name_tag + list(component_tags)
        
        # Append results
        addresses.append(address)
        tags.append(address_tags)
    
    return addresses, tags

In [85]:
random_addresses, random_tags = generate_address(10)

# Convert the generated addresses and tags to a DataFrame for CSV export
df_addresses = pd.DataFrame({
    "Address": random_addresses,
    "Tags": random_tags
})

In [87]:
df_addresses

,Address,Tags
0,นายพัฒน์พงศ์ สวัสดี นนทบุรี ห้วยขวาง สีกัน 101...,"[O, O, LOC, LOC, LOC, POST, ADDR]"
1,นายสมชาย มีสุข 96/28 สระบุรี ศาลายา บางเขน 10100,"[O, O, ADDR, LOC, LOC, LOC, POST]"
2,นายกิตติ สุวรรณ 10120 604/36 บางกอกใหญ่ บางนา ...,"[O, O, POST, ADDR, LOC, LOC, LOC]"
3,นายกิตติ ทรงชัย กทม ลาดพร้าว 345/14 10150 บางมด,"[O, O, LOC, LOC, ADDR, POST, LOC]"
4,นายวัฒนา สวัสดี ปทุมธานี 997/49 10540 ลาดพร้าว...,"[O, O, LOC, ADDR, POST, LOC, LOC]"
5,นายสมชาย เจริญสุข 234/99 10240 ห้วยขวาง สวนหลว...,"[O, O, ADDR, POST, LOC, LOC, LOC]"
6,นายสมศักดิ์ กิตติกูล อ่างทอง 364/27 บางรัก พญา...,"[O, O, LOC, ADDR, LOC, LOC, POST]"
7,นายสมศักดิ์ สุวรรณ 10270 อ่างทอง 389/35 คลองตั...,"[O, O, POST, LOC, ADDR, LOC, LOC]"
8,นายอรุณ มีสุข กทม ดอนเมือง 825/41 บางรัก 10230,"[O, O, LOC, LOC, ADDR, LOC, POST]"
9,นายสุริยะ รุ่งเรือง 937/83 10120 ปทุมธานี ลาดย...,"[O, O, ADDR, POST, LOC, LOC, LOC]"


In [122]:
def get_random_ex():
    return df_addresses.sample(n=1).iloc[0]

In [129]:
print(get_random_ex())

Address    นายวัฒนา สวัสดี ปทุมธานี 997/49 10540 ลาดพร้าว...
Tags                       [O, O, LOC, ADDR, POST, LOC, LOC]
Name: 4, dtype: object


In [ ]:
# Shapley
cbr = joblib.load('catboost.joblib')

# Data Cleansing
tokens = long_text.split()
feature_matrix = [tokens_to_features(tokens, i) for i in range(len(tokens))]  # Extract features
feature_df = pd.DataFrame(feature_matrix)
feature_df['BOS'] = feature_df['BOS'].apply(lambda x: 1 if x else 0)
feature_df['EOS'] = feature_df['EOS'].apply(lambda x: 1 if x else 0)
cat_features = feature_df.select_dtypes(include=['object']).columns.tolist()
feature_df_new = feature_df.apply(lambda row: fill_values(row, cat_features), axis=1)
# Prediction
predicted_probs = cbr.predict(feature_df_new)
# Shap
explainer = shap.Explainer(cbr)
shap_values = explainer(feature_df_new)
classes = cbr.classes_
#Waterfall plot
instance_idx = 0  # Only one instance, so index is 0
class_idx = i     # Change to the index of the class you want to explain (e.g., "ADDR")
    
    # Extract SHAP values and base value
shap_values_for_class = shap_values[instance_idx].values[:, class_idx]
base_value_for_class = shap_values[instance_idx].base_values[class_idx]
data_for_instance = shap_values[instance_idx].data
    
    # Generate waterfall plot
shap.plots.waterfall(
    shap.Explanation(
        values=shap_values_for_class,
        base_values=base_value_for_class,
        data=data_for_instance,
        feature_names=feature_df_new.columns
        )
    )
plt.show()